# AD Group members

This notebook will get the members of the AD groups 
There are two ways of doing it and a number of code blocks below

- one via the AD Module which needs RSAT installed on your laptop 
- One via dbatools
- With dbatools you can also get a login that is able to connect to an instance and see which group it gets access via
- You can also create Excel workbooks for either  


Run this code first - always - It helps to set the output

In [ ]:
If(Get-Module ActiveDirectory -ListAvailable ){
    Write-Output "you have the Active Directory module so you can use the Active Directory code"
}
if(Get-Module dbatools -ListAvailable){
    Write-Output "you have the dbatools module so you can use the dbatools code"
}
Write-Output "Your choices are presented above - if there is nothing, you will need to install the dbatools module"
# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

## Using the Active Directory Module

Add the GAD group that you wish to get the members for and run the code

You need to enter the group name here

In [ ]:
$ADGroup = '' # Just the group name after the \
Write-Output "Members of $ADGroup"
Get-ADGroupMember -Identity $ADGroup | Select Name, SamAccountName

## Using dbatools

With dbatools you can get all of users that have access to an instance and the groups that they get access via.

You need to enter the instance here



In [ ]:
$SQLInstance = ''
Find-DbaLoginInGroup -SqlInstance $SQLInstance | Format-Table

## Using dbatools for a single user

You can find which groups a user used to gain access 

In [ ]:
$SQLInstance = ''
$Login = '\'
Find-DbaLoginInGroup -SqlInstance $SQLInstance -Login $Login | Format-Table

# Creating Excel

You can create Excel sheets if you wish - This requires that the ImportExcel module has been installed

## Active Directory



In [26]:
#Check for modules and install
$Modules = 'dbatools', 'ImportExcel'

if((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted'){
    Write-Output "             #######################################################"
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    Write-Output "             ######     Please follow the Setting Up notebook ######"
} else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
$Modules.ForEach{
    If(-not(Get-Module $psitem -ListAvailable)){
        Write-Output "             #######################################################"
        Write-Output "                        We don't have the $psitem module."
        Write-Output "             ######     Please follow the Setting Up notebook ######"
    }
    else {
        Write-Output "We have the $psitem module already"
    }
}

The PowerShell Gallery is trusted I will continue
We have the dbatools module already
We have the ImportExcel module already


In [ ]:
$ADGroups = '', '' # comma seperated list - Just the group name after the \
$Directory = 'C:\temp' # the directory you wish the excel sheet create in
$Path = $Directory + '\' + $ADGroups + '_Members_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'
Write-Output "Members of $ADGroups"
$ADGroups | ForEach-Object { $group = $psitem; Get-ADGroupMember $group | Select @{Name='Group';Exp={$group}}, Name, SamAccountName } | Export-Excel -Path $Path -AutoSize -FreezeTopRow -AutoFilter
Invoke-Item $Path

## dbatools

In [ ]:
$SQLInstance = '' # enter the SQL instance here
$Directory = 'C:\temp' # the directory you wish the excel sheet create in
$Path = $Directory + '\Instance' + $SQLInstance + '_Members_and_Groups' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'
Find-DbaLoginInGroup -SqlInstance $SQLInstance | Select-Object SqlInstance, Login, DisplayName, MemberOf, ParentADGroupLogin |Export-Excel -Path $Path -AutoSize -FreezeTopRow -AutoFilter
Invoke-Item $Path